In [ ]:
import pickle
import pandas as pd
import scipy
import numpy as np
import json
import os
import s3fs
from maayanlab_bioinformatics.dge import limma_voom_differential_expression
from maayanlab_bioinformatics.normalization import quantile_normalize, zscore_normalize
from tqdm import tqdm
import sys
import contextlib
@contextlib.contextmanager
def suppress_output(stdout=True, stderr=True, dest='/dev/null'):
    ''' Usage:
    with suppress_output():
        print('hi')
    '''
    dev_null = open(dest, 'a')
    if stdout:
        _stdout = sys.stdout
        sys.stdout = dev_null
    if stderr:
        _stderr = sys.stderr
        sys.stderr = dev_null
    try:
        yield
    finally:
        if stdout:
            sys.stdout = _stdout
        if stderr:
            sys.stderr = _stderr

In [ ]:
endpoint = 'https://d2h2.s3.amazonaws.com/'
base_url = 'data'
s3 = s3fs.S3FileSystem(anon=True, client_kwargs={'endpoint_url': endpoint})

version = "v1.1"

In [ ]:
with open(f'../app/static/searchdata/metadata-v1.pickle', 'rb') as f:	
		gse_metadata = pickle.load(f)

In [ ]:
def compute_sigs(expr_df, metadata_df, species, gse):
    
    ctrl_conditions = []
    metadata_df = metadata_df[metadata_df.index.isin(expr_df.columns.values)]
    all_conditions = list(set(metadata_df['Condition'].values))
    conditions = []
    
    # find conditions with replicates
    for condition in all_conditions:
        if len(metadata_df[metadata_df['Condition'] == condition]) > 1:
            conditions.append(condition)

    if len(conditions) >= 1:
        # identify control conditions or use first condition as default
        for condition in conditions:
            split_conditions =condition.lower().replace('-', ' ').replace('_', ' ').split(' ')
            if 'control' in split_conditions or 'healthy' in split_conditions:
                ctrl_conditions.append(condition)

        if len(ctrl_conditions) < 1:
            ctrl_conditions = conditions

        seen = []
        for ctrl_cond in ctrl_conditions:
            samps_condition = metadata_df[metadata_df['Condition'] == ctrl_cond].index.values
            for condition in conditions:
                if condition != ctrl_cond and {ctrl_cond, condition} not in seen:
                    seen.append({ctrl_cond, condition})
                    samps_condition_2 = metadata_df[metadata_df['Condition'] == condition].index.values
                    group = list(metadata_df[metadata_df['Condition'] == condition]['Group'].values)[0]
                    sig_name = f"{gse}-{ctrl_cond}-vs-{condition.replace('/', ' ')}-{species}-{group}"
                    if not os.path.exists(f'{species.lower()}/{sig_name}.tsv'):
                        try:
                            with suppress_output():
                                dge = limma_voom_differential_expression(
                                    expr_df[samps_condition], expr_df[samps_condition_2],
                                    voom_design=True,
                                )
                            dge.to_csv(f'{species.lower()}/{sig_name}.tsv', sep='\t')
                        except:
                            print("error computing:", sig_name)

In [ ]:
os.makedirs('human', exist_ok=True)
os.makedirs('mouse', exist_ok=True)
os.makedirs('signatures', exist_ok=True)
os.makedirs(f'signatures/{version}', exist_ok=True)

human_done = set(map(lambda x: x.split('-')[0], os.listdir('human')))
mouse_done = set(map(lambda x: x.split('-')[0], os.listdir('mouse')))
signatures_computed = {'human': human_done, 'mouse': mouse_done}

In [ ]:
species_list = ['human', 'mouse']
for species in species_list:
    for gse in tqdm(gse_metadata[species]):
        if gse not in signatures_computed[species]:
            metadata_file = f"{base_url}/{species}/{gse}/{gse}_Metadata.txt"
            expr_file = f"{base_url}/{species}/{gse}/{gse}_Expression.txt"

            metadata_df = pd.read_csv(s3.open(metadata_file), sep='\t', index_col=0)
            expr_df = pd.read_csv(s3.open(expr_file), sep='\t', index_col=0)
            expr_df.dropna(inplace=True)
            compute_sigs(expr_df, metadata_df, species, gse)


In [ ]:

species_list = ['human', 'mouse']

for species in species_list:
    rna = []
    micro = []
    for gse in tqdm(gse_metadata[species]):
        sig_files = list(filter(lambda x: x.split('-')[0] == gse, os.listdir(species)))
        if gse_metadata[species][gse]['type'][0] == 'Expression profiling by array':
            for sig in sig_files:
                s = pd.read_csv(f'{species}/{sig}', sep='\t', index_col=0)['logFC'].astype(np.float64)
                s.name = sig.replace('.tsv', '')
                micro.append(s)
        elif gse_metadata[species][gse]['type'][0] == 'Expression profiling by high throughput sequencing':
            for sig in sig_files:
                s = pd.read_csv(f'{species}/{sig}', sep='\t', index_col=0)['logFC'].astype(np.float64)
                s.name = sig.replace('.tsv', '')
                rna.append(s)
        else:
            print(gse, gse_metadata[species][gse]['type'][0])

    micro_df = pd.concat(micro, axis=1)
    micro_df.to_csv(f'signatures/{version}/{species}-micro.csv')
    micro_df = micro_df.apply(lambda x: np.abs(x))
    micro_df_ranks = micro_df.apply(lambda x: x.rank(ascending=False)).T
    micro_df_ranks = micro_df_ranks.reset_index()
    micro_df_ranks.to_feather(f'signatures/{version}/{species}_micro_fc_signrank.f')

    rna_df = pd.concat(rna, axis=1)
    rna_df.to_csv(f'signatures/{version}/{species}-rna.csv')
    rna_df = rna_df.apply(lambda x: np.abs(x))
    rna_df_ranks = rna_df.apply(lambda x: x.rank(ascending=False)).T
    rna_df_ranks = rna_df_ranks.reset_index()
    rna_df_ranks.columns = (rna_df_ranks.columns.values).astype(str)
    rna_df_ranks.to_feather(f'signatures/{version}/{species}_rna_fc_signrank.f')


In [ ]:
with open('signature_idx.json') as f:
    signature_idx = json.load(f)

## Normalize and Z-score rows

In [ ]:
species_list = ['human', 'mouse']

for species in species_list:
    for t in ['micro', 'rna']:
        df = pd.read_csv(f'signatures/{version}/{species}-{t}.csv', index_col=0).astype('float64')
        df = df.loc[signature_idx[f'{species}_{t}']]
        df_vals= df.fillna(0)
        print('norming')
        df = quantile_normalize(df_vals, axis=0)
        print('done norming')
        df.to_csv(f'signatures/{version}/{t}_{species}_fc.csv')
        df.T.reset_index().to_feather(f'signatures/{version}/{t}_{species}_fc.f')
        df = zscore_normalize(df)
        df = df.apply(lambda x: scipy.stats.norm.sf(np.abs(x)))
        df.to_csv(f'signatures/{version}/{t}_{species}_pval.csv')
        df.T.reset_index().to_feather(f'signatures/{version}/{t}_{species}_pval.f')